In [1]:
"""Accruals

Demonstration of how to model accruals in LUSID

Attributes
----------
transaction configuration
cocoon
holdings
"""

'Accruals\n\nDemonstration of how to model accruals in LUSID\n\nAttributes\n----------\ntransaction configuration\ncocoon\nholdings\n'

# Accruals

This notebook demonstrates how accruals can be modelled in LUSID. We show how we can book a management fee that is due to be paid at the end of the month and track the accrual separately from the main cash balance. At the end of the month we see the fee being deducted from the cash balance.

In [2]:
# Import lusid specific packages
import lusid
import lusid.models as models
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon import identify_cash_items
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
import fbnsdkutilities.utilities as utils

import os
import pandas as pd
pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = utils.ApiClientFactory(
    lusid,
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print('LUSID Environment Initialised')
print('LUSID version : ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID version :  0.6.11251.0


# Transaction configuration

Define a transaction configuration for the accrual, here we set the accrual account increase which will then be added to the cash balance on settlement.

In [3]:
def upsert_transaction_config(config):
    config_api = api_factory.build(lusid.api.SystemConfigurationApi)
    txn_types = config_api.list_configuration_transaction_types()

    configs = set([
        (aliases.type, aliases.transaction_group)
        for txn_config in txn_types.transaction_configs
        for aliases in txn_config.aliases
    ])

    aliases = set([(a.type, a.transaction_group) for a in config.aliases])

    if configs.issuperset(aliases):
        return

    config_api.create_configuration_transaction_type(transaction_configuration_data_request=config)

accrual_config = models.TransactionConfigurationDataRequest(
    aliases=[
        models.TransactionConfigurationTypeAlias(
            type="MgmtFee",
            description="Management fee accrual",
            transaction_class="Basic",
            transaction_group="default",
            transaction_roles="Longer"
        ),
    ],
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="CashAccrual",
            side="Side2",
            direction=-1
        )
    ]
)
upsert_transaction_config(accrual_config)

Create a scope and portfolio code

In [4]:
scope = "accruals-demo"
portfolio_code = "EQUITY_UK"

In this portfolio we add a cash injection and book a transaction for a stock purchase. We also book an accrual for a management fee that is paid at the end of the month

In [5]:
df = pd.read_csv("data/accruals/equity_transactions.csv")
df

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,cash_transactions
0,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0001,Buy,01/04/2020,03/04/2020,120000,5,600000,GBP,NaN
1,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GBP,GBP,cash,GBP,GBP Cash,cash_001,FundsIn,01/04/2020,03/04/2020,1000000,1,1000000,GBP,GBP
2,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GBP,GBP,cash,GBP,GBP Cash,cash_002,MgmtFee,01/04/2020,30/04/2020,600,1,600,GBP,GBP


In [6]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Sedol": "sedol",
    },
    "required": {
        "name": "name"
    },
    "cash_flag": {
        "cash_identifiers": {
            "cash_transactions" : ["GBP"]
        },
        "implicit": "currency"
    }
}

In [7]:
instr_df, mapping = identify_cash_items(
    dataframe = df.copy(),
    mappings = instrument_mapping,
    file_type = "instruments",
    remove_cash_items = True
)

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=instr_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])


,success,failed,errors
0,1,0,0


## Create portfolio

In [8]:
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "$GBP",
    },
    "optional": {"created": "$2020-01-01T00:00:00+00:00"},
}

In [9]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=[],
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


# Upload transactions

In [10]:
transaction_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Currency": "cash_transactions"
    },
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "portfolio_base_currency",
        "settlement_date": "txn_settle_date",
    },
    "optional": {},
    "properties": [],
}

In [11]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope=scope,
)

succ, failed = format_transactions_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


# Get holdings

In [12]:
# transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
transaction_portfolio_api = lusid.api.TransactionPortfoliosApi(api_factory.build(lusid.api.TransactionPortfoliosApi))

def get_holdings(effective_at):
    response = transaction_portfolio_api.get_holdings(
        scope=scope,
        code=portfolio_code,
        effective_at=effective_at,
        property_keys=["Instrument/default/Name"]
    )

    holdings_df = lusid_response_to_data_frame(response, rename_properties=True)
    display(holdings_df)

Getting the holdings before settlement shows:
* the holding in the purchased stock (P)
* the committed cash against the purchase (C)
* the received cash injection (R)
* the accrual from the fee (A)

In [13]:
get_holdings("2020-04-01")

,instrument_scope,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency,holding_type_name,transaction.transaction_id,transaction.type,transaction.instrument_identifiers.Instrument/default/ClientInternal,transaction.instrument_scope,transaction.instrument_uid,transaction.transaction_date,transaction.settlement_date,transaction.units,transaction.transaction_price.price,transaction.transaction_price.type,transaction.total_consideration.amount,transaction.total_consideration.currency,transaction.exchange_rate,transaction.transaction_currency,transaction.properties.Transaction/default/ResultantHolding.key,transaction.properties.Transaction/default/ResultantHolding.value.metric_value.value,transaction.entry_date_time,transaction.transaction_status,transaction.cancel_date_time,transaction.instrument_identifiers.Instrument/default/Currency,transaction.properties
0,default,LUID_KR3A1NMI,{},Aviva,EQUITY_UK,accruals-demo,P,120000.0,0.0,600000.0,GBP,600000.0,GBP,GBP,Position,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,default,CCY_GBP,{},GBP,EQUITY_UK,accruals-demo,C,-600000.0,0.0,-600000.0,GBP,-600000.0,GBP,GBP,CashCommitment,trd_0001,Buy,EQ_1234,default,LUID_KR3A1NMI,2020-04-01 00:00:00+00:00,2020-04-03 00:00:00+00:00,120000.0,5.0,Price,600000.0,GBP,1.0,GBP,Transaction/default/ResultantHolding,120000.0,2023-04-24 16:48:25.272273+00:00,Active,0001-01-01 00:00:00+00:00,NaN,NaN
2,default,CCY_GBP,{},GBP,EQUITY_UK,accruals-demo,A,1000000.0,0.0,1000000.0,GBP,1000000.0,GBP,GBP,Accrual,cash_001,FundsIn,NaN,default,CCY_GBP,2020-04-01 00:00:00+00:00,2020-04-03 00:00:00+00:00,1000000.0,1.0,Price,1000000.0,GBP,1.0,GBP,NaN,NaN,2023-04-24 16:48:25.272273+00:00,Active,0001-01-01 00:00:00+00:00,GBP,{}
3,default,CCY_GBP,{},GBP,EQUITY_UK,accruals-demo,A,-600.0,0.0,-600.0,GBP,-600.0,GBP,GBP,Accrual,cash_002,MgmtFee,NaN,default,CCY_GBP,2020-04-01 00:00:00+00:00,2020-04-30 00:00:00+00:00,600.0,1.0,Price,600.0,GBP,1.0,GBP,NaN,NaN,2023-04-24 16:48:25.272273+00:00,Active,0001-01-01 00:00:00+00:00,GBP,{}


After the stock has settled we can see the cash balance updated and the fee accrual still there.

In [14]:
get_holdings("2020-04-06")

,instrument_scope,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency,holding_type_name,transaction.transaction_id,transaction.type,transaction.instrument_identifiers.Instrument/default/Currency,transaction.instrument_scope,transaction.instrument_uid,transaction.transaction_date,transaction.settlement_date,transaction.units,transaction.transaction_price.price,transaction.transaction_price.type,transaction.total_consideration.amount,transaction.total_consideration.currency,transaction.exchange_rate,transaction.transaction_currency,transaction.properties,transaction.entry_date_time,transaction.transaction_status,transaction.cancel_date_time
0,default,LUID_KR3A1NMI,{},Aviva,EQUITY_UK,accruals-demo,P,120000.0,120000.0,600000.0,GBP,600000.0,GBP,GBP,Position,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,default,CCY_GBP,{},GBP,EQUITY_UK,accruals-demo,B,400000.0,400000.0,400000.0,GBP,400000.0,GBP,GBP,Balance,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
2,default,CCY_GBP,{},GBP,EQUITY_UK,accruals-demo,A,-600.0,0.0,-600.0,GBP,-600.0,GBP,GBP,Accrual,cash_002,MgmtFee,GBP,default,CCY_GBP,2020-04-01 00:00:00+00:00,2020-04-30 00:00:00+00:00,600.0,1.0,Price,600.0,GBP,1.0,GBP,{},2023-04-24 16:48:25.272273+00:00,Active,0001-01-01 00:00:00+00:00


At the end of the month the accrual is paid out and the cash balance updated

In [15]:
get_holdings("2020-04-30")


,instrument_scope,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency,holding_type_name
0,default,LUID_KR3A1NMI,{},Aviva,EQUITY_UK,accruals-demo,P,120000.0,120000.0,600000.0,GBP,600000.0,GBP,GBP,Position
1,default,CCY_GBP,{},GBP,EQUITY_UK,accruals-demo,B,399400.0,399400.0,399400.0,GBP,399400.0,GBP,GBP,Balance
